In [71]:
! pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
import os
import pandas as pd
import requests
from selenium.webdriver.common.keys import Keys
from time import sleep
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from retry import retry

In [57]:
driver = webdriver.Remote(
   command_executor='http://selenium-hub:4444/wd/hub',
   desired_capabilities=DesiredCapabilities.CHROME)

<ipython-input-57-eb4b3c9f8e7a>:1: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  driver = webdriver.Remote(


In [58]:
urls = pd.read_csv("url_list.csv")

In [59]:
urls = urls["url"].to_list()

In [60]:
urls

['https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=5126',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=9560',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=5125',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=7388',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=5038',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=9558',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=2935',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=7110',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=5036',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=9557',
 'https://www2.jpx.co.jp/tseHpFront/StockSearch.do?method=topsearch&topSearchStr=5034',
 'https://www2.jpx.co.jp/tseHpFr

In [51]:
company = driver.find_element(By.XPATH,
                               '//*[@id="body_basicInformation"]/div/div[3]/h3')
company = company.text

In [61]:
driver.get("https://www2.jpx.co.jp/tseHpFront/JJK010030Action.do")

In [65]:
#  スクレイピング定義
def _get_info(url):
    
    driver.get(url)
    time.sleep(1.0)
    driver.find_element(By.CLASS_NAME, 'activeButton').click()
    time.sleep(1.0)
    
    ## 社名
    company = driver.find_element(By.XPATH,
                               '//*[@id="body_basicInformation"]/div/div[3]/h3')
    company = company.text
    ## コード
    code = driver.find_element(By.XPATH,
                               '//*[@id="body_basicInformation"]/div/table[1]/tbody/tr[2]/td[1]')
    code = code.text
    ## ISINコード
    ISIN = driver.find_element(By.XPATH,
                               '//*[@id="body_basicInformation"]/div/table[1]/tbody/tr[2]/td[2]')
    ISIN = ISIN.text
    ## 設立年月日
    birth = driver.find_element(By.XPATH,
                               '//*[@id="body_basicInformation"]/div/table[2]/tbody/tr[4]/td[1]')
    birth = birth.text
    ## 業種
    industry = driver.find_element(By.XPATH,
                               '//*[@id="body_basicInformation"]/div/table[1]/tbody/tr[2]/td[4]')
    industry = industry.text
    ## 代表者役職
    position = driver.find_element(By.XPATH, 
                                       '//*[@id="body_basicInformation"]/div/table[2]/tbody/tr[8]/td[2]'
                                  )
    position = position.text
    ## 代表者氏名
    representitive = driver.find_element(By.XPATH, 
                                        '//*[@id="body_basicInformation"]/div/table[2]/tbody/tr[8]/td[3]'
                                        )
    representitive = representitive.text
    ## 上場株式数
    num_of_stocks_listed = driver.find_element(By.XPATH, 
                                       '//*[@id="body_basicInformation"]/div/table[3]/tbody/tr[2]/td[2]'
                                  )
    num_of_stocks_listed = num_of_stocks_listed.text
    ## 発行済株式数
    num_of_stocks_issued = driver.find_element(By.XPATH,
                                              '//*[@id="body_basicInformation"]/div/table[3]/tbody/tr[2]/td[3]'
                                              )
    num_of_stocks_issued = num_of_stocks_issued.text
    ## 
    
    data = pd.DataFrame(
    columns=['company', 'code', 'ISIN', "establishment","industry", "position", 
             "representitive", "num_of_stocks_listed", "num_of_stocks_issued",],
    data={'company':[company],'code':[code], 'ISIN':[ISIN],'establishment':[birth],
          'industry':[industry], 'position':[position], 'representitive':[representitive],
          'num_of_stocks_listed':[num_of_stocks_listed],"num_of_stocks_issued":[num_of_stocks_issued],
         }
    )
    
    return data

##  エラーURLはスキップ
@retry(tries=10, delay=1) # 失敗したときにリトライする
def get_info(url):
    # 失敗したら何も返さない
    try:
        df = _get_info(url)
        return df
    except Exception as e:
        return None

In [66]:
#  上のコードはこれと一緒
dfs = []
# tqdmで進捗バーを出す
for url in tqdm(urls):
    # 各urlからとってデータをdfに格納
    df = get_info(url)
    #  それをdfsに追加する
    dfs.append(df)
    # 最後に同じ列同士をconcatで結合
df_all = pd.concat(dfs)

  0%|          | 0/496 [00:00<?, ?it/s]

In [73]:
df_all.to_csv("company.csv")